In [1]:
import numpy as np
import math
from   more_itertools import powerset 
import pandas as pd
from   ucimlrepo import fetch_ucirepo
from   sklearn.model_selection import train_test_split
from   sklearn.linear_model import LinearRegression


In [2]:
# Load and split the data
wine_quality = fetch_ucirepo(id=186)
wine_subset  = wine_quality['data']['original'][wine_quality['data']['original']['color'] == 'white']
X = np.array(wine_subset[['fixed_acidity', 'volatile_acidity', 'citric_acid','residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol']])
y = np.array(wine_subset['quality'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
player_names = ['fixed_acidity', 'volatile_acidity', 'citric_acid','residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol']
N = set([i for i in range(X.shape[1])])

In [4]:
# Standardise the training data
X_mu, X_sigma, y_mu, y_sigma = X_train.mean(axis=0), X_train.std(axis=0), y_train.mean(axis=0), y_train.std(axis=0)

X_train = (X_train - X_mu) / X_sigma
y_train = (y_train - y_mu) / y_sigma

X_test = (X_test - X_mu) / X_sigma

$$X^TXw = X^Ty$$ 
$$w=(X^TX)^{-1}X^Ty$$

In [5]:
w = np.matmul(np.matmul(np.linalg.inv(np.matmul(X_train.T, X_train)), X_train.T), y_train)

$$R^2  = \frac{1}{N}w^TX^TXw$$
$$R^2_i = \frac{1}{N}w_i (X^TXw)$$

In [6]:
R2  = np.matmul(w.T,np.matmul(np.matmul(X_train.T, X_train), w))/len(y_train)
R2i = np.matmul(np.diag(w),np.matmul(np.matmul(X_train.T, X_train), w))/len(y_train)
print(R2)
print(R2i) 

0.2843126626414035
[-5.15764443e-03  4.48079867e-02  1.16066061e-04 -3.99818409e-02
  1.29593436e-04  2.77336079e-03  1.86662385e-03  1.27245440e-01
  1.04933860e-02  5.15467153e-03  1.36865019e-01]


$$\hat{y} = Xw$$
$$MSE = \frac{1}{N}\sum(y-\hat{y})^2$$

In [7]:
yh  = np.matmul(X_test, w)
MSE = np.matmul((y_test - yh).T, (y_test - yh)) / len(y_test)

In [8]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [9]:
model.coef_

array([ 0.04371106, -0.2193717 , -0.00827766,  0.41280525, -0.00065067,
        0.09626305, -0.01148575, -0.42336479,  0.10171807,  0.08312901,
        0.3170512 ])

In [10]:
w

array([ 0.04371106, -0.2193717 , -0.00827766,  0.41280525, -0.00065067,
        0.09626305, -0.01148575, -0.42336479,  0.10171807,  0.08312901,
        0.3170512 ])

In [47]:
def predict(idx):
    X_vector   = X_test[idx]
    y_standard = np.matmul(X_vector, w)
    y_actual   = (y_standard * y_sigma) + y_mu
    return y_actual

E_f = np.mean(predict([i for i, _ in enumerate(X_test)]))

def value(xi, S, sample_size):
    s_idx    = np.random.randint(low=0, high=len(X_train), size=sample_size)
    X_sample = X_train[s_idx]   
    X_sample[:,list(S)] = X_test[xi,list(S)]
    y_standard = np.matmul(X_sample, w)
    y_actual   = (y_standard * y_sigma) + y_mu
    return np.mean(y_actual) - E_f
   
def marginal(xi, j, S, sample_size):
    return value(xi, S.union(j), sample_size) - value(xi, S, sample_size)

def gamma(N,S):
    return math.factorial(len(S)) * math.factorial(len(N) - len(S) - 1) / math.factorial(len(N))

def phi(xi, j, N, sample_size):
    players = N - j
    return np.sum([gamma(N, S) * marginal(xi, j, set(S), sample_size) for S in powerset(players)])


In [55]:
idx   = 0
phi_T = E_f
E_y   = np.mean(y_train)
for player in N:
    phi_i = (phi(idx,{player}, N, 4000) - E_y) * y_sigma
    phi_T += phi_i
    print('phi_'+ player_names[player], round(phi_i,3))
    
print('Total phi', round(phi_T,3))
print('Total reward', round(predict(idx) , 2))

phi_fixed_acidity -0.035
phi_volatile_acidity -0.018
phi_citric_acid -0.006
phi_residual_sugar 0.274
phi_chlorides -0.0
phi_free_sulfur_dioxide 0.09
phi_total_sulfur_dioxide -0.002
phi_density 0.043
phi_pH -0.054
phi_sulphates 0.057
phi_alcohol 0.094
Total phi 6.334
Total reward 6.37


In [56]:
np.mean(predict([i for i, _ in enumerate(X_test)]))
    

5.89318843511457

In [46]:
E_f

5.89318843511457

In [31]:
(E_y * y_sigma) + y_mu

5.871362940275651